# Multiple Regression (gradient descent)

We will use sklearn along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. We will:
* Add a constant column of 1's to a pandas dataframe to account for the intercept
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

In [57]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

## Load house sales data
Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

Read data from csv file and put in dataframe.

In [2]:
sales = pd.read_csv('data/kc_housing_sales_data.csv')
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
# print the shape of the DataFrame
sales.shape

(21613, 21)

## Convert to Numpy Array
The predicted value given the weights and the features is just the dot product between the feature and weight vector.

Similarly, if we put all of the features row-by-row in a matrix then the predicted value for all the observations can be computed by right multiplying the "feature matrix" by the "weight vector".

Now we will write a function that will accept a dataframe, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

In [45]:
def get_numpy_data(data_frame, features, output):
    
    data_frame['constant'] = 1 # this is how you add a constant column to a data frame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
   
    # select the columns of data_frame given by the features list into the data frame features_frame (now including constant):
    features_frame = data_frame[features]
       
    # the following line will convert the features_frame into a numpy matrix:
    #feature_matrix = features_frame.to_numpy()
    feature_matrix = features_frame.as_matrix()
       
    # assign the column of data_frame associated with the output to the array output_array
    output_array = data_frame[output]
    
    # the following will convert the array into a numpy array by first converting it to a list
    output_array = output_array.as_matrix()
    
    return(feature_matrix, output_array)

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [48]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0] # and the corresponding output

[   1 1180]
221900.0


## Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrayws we can use np.dot() to compute this:

In [49]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print predicted_value

1181.0


np.dot() also works when dealing with a matrix and a vector. The predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features matrix and the weights vector. 

Predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [50]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

Let's test the above code:

In [51]:
test_predictions = predict_output(example_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


## Computing the Derivative

Let's compute the derivative of the regression cost function. Cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... + w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... + w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors.

Following is the derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [52]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    
    derivative = 2*np.dot(errors,feature)
    
    return(derivative)

Lets test above feature derivartive:

In [54]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 

my_weights = np.array([0., 0.]) # this makes all the predictions 0
predictions = predict_output(example_features, my_weights) 

# just like data_frames 2 numpy arrays can be elementwise subtracted with '-': 
errors = predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"

derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850016.0
-23345850016.0


## Gradient Descent

Lets now write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of increase and therefore the negative gradient is the direction of decrease and we're trying to minimize a cost function.

The amount by which we move in the negative gradient direction is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

Following is the gradient descent function below using derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria.

In [55]:
from math import sqrt

In [56]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
        
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    
    while not converged:
        
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        
        # compute the errors as predictions - output
        errors = predictions - output
        
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        
        for i in range(len(weights)): # loop over each weight
            
            # feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])
            
            # add the squared value of the derivative to the gradient magnitude (for assessing convergence)
            
            gradient_sum_squares = gradient_sum_squares + (derivative * derivative)
                        
            # subtract the step size times the derivative from the current weight
            weights[i] = weights[i] - (step_size * derivative)
            
        # compute the square-root of the gradient sum of squares to get the gradient matnigude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
        
    return(weights)

Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features.

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

## Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [58]:
train_data,test_data = train_test_split(sales, test_size = 0.2)

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimat the parameters in the simple regression on squarefeet. 

Now set up the feature_matrix, output, initial weights and step size for the first model:

In [61]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'

(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)

initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

/home/osboxes/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Now run gradient descent with the above parameters.

In [62]:
weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
print weights

[-46999.8891626     283.39752735]


Use newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (we will need to create a numpy array of the test feature_matrix and test output first:

In [63]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

/home/osboxes/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Lets now compute predictions using test_simple_feature_matrix and weights from above.

In [64]:
model1_predictions = predict_output(test_simple_feature_matrix, weights)
model1_predictions

array([ 531131.06662224,  545300.94298952,  845702.32197576, ...,
        227895.71236254,  182552.10798726,  601980.44845862])

### RSS - Residual Sum of Squares
Now use your predictions and the output to compute the RSS for model 1 on TEST data.

In [66]:
errors = model1_predictions - test_output
errors_squared = errors * errors
rss_model1 = sum(errors_squared)
print rss_model1

2.84024859495e+14


## Running a multiple regression

Now lets use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [67]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'

(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)

initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

/home/osboxes/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Use the above parameters to estimate the model weights.

In [68]:
weights_model2 = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
print weights_model2

[ -9.99999832e+04   2.48294211e+02   6.31795751e+01]


Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [69]:
(test_model_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
model2_predictions = predict_output(test_model_feature_matrix, weights_model2)
model2_predictions

/home/osboxes/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


array([ 609958.43955834,  556666.39201124,  881142.4437639 , ...,
        234351.17286308,  216105.15460209,  629069.88129436])

### RSS - Residual Sum of Squares
Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [70]:
errors_model2 = model2_predictions - test_output
errors_model2_squared = errors_model2 * errors_model2
rss_model2 = sum(errors_model2_squared)
print rss_model2

2.78323587281e+14
